<a href="https://colab.research.google.com/gist/ruvnet/0a609e26512b32ac0a8e772513be0d8e/agentchat_multi_task_async_chats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iterative Book Writing with AI\n\nThis notebook demonstrates an approach for using AI to iteratively generate the structure and content of a book:\n\n1. Generate an initial high-level outline of the book, with suggested parts and chapters. \n\n2. Review the outline and make any desired changes. The human author can modify, add or remove parts and chapters as needed.\n\n3. For each chapter, generate a more detailed outline of the key points to cover.\n\n4. Review the detailed chapter outlines and iterate as needed based on human feedback. \n\n5. Begin drafting the content for each chapter, using the detailed outlines as a guide. Generate a few paragraphs at a time.\n\n6. After each new content generation, review the output and revise as needed. The human author can modify the text directly.\n\n7. Continue the process of content generation and human feedback until a full first draft of each chapter is produced. \n\n8. Review the full first draft of the book. Look for opportunities to improve flow and fill in any gaps.\n\n9. Repeat the revision process as many times as needed, generating new text and letting the human author provide input.\n\n10. Once the book content is finalized, generate front and back matter like an introduction, acknowledgments, references, and index.\n\nKey aspects:\n- Start with a high-level outline and iteratively add more detail \n- Generate content in small chunks and frequently get human input\n- Use the evolving book content itself to inform subsequent generation steps\n- Allow plenty of cycles of revision based on human feedback \n- Continue until the author is fully satisfied with the final book\n\nThis iterative approach, alternating between AI content generation and human review, can help efficiently create a well-structured book while keeping the author in control of the final product.\n\n:::info Requirements\n\nInstall `pyautogen`:\n```bash\npip install pyautogen\n```\n\nFor more information, please refer to the [installation guide](/docs/installation/).\n\n:::

In [ ]:
import autogen\n\nconfig_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")\nllm_config = {"config_list": config_list}

:::tip\n\nLearn more about the various ways to configure LLM endpoints [here](/docs/topics/llm_configuration).\n\n:::

### Example Book Writing Tasks\nBelow are the key tasks involved in the iterative book writing process:\n

In [ ]:
book_tasks = [\n    """Generate a high-level outline for a non-fiction book on the history of artificial intelligence, with suggested parts and chapters.""",\n    """Review the book outline and make any desired changes to the structure and chapter breakdown.""", \n    """For each chapter in the outline, generate a more detailed outline of the key topics and points to cover.""",\n    """Review the detailed chapter outlines and provide feedback. Ensure the flow between chapters makes sense.""",\n    """Begin drafting the content for each chapter, using the detailed outlines as a guide. Generate a few paragraphs at a time.""",\n    """Review each content generation and revise as needed. Modify the text directly where required.""",\n    """Continue the process of content generation and human feedback until a full first draft of each chapter is produced.""", \n    """Review the full first draft of the book. Identify opportunities to improve flow, fill gaps, and optimize the structure.""",\n    """Repeat the revision process as many times as needed, generating new text and incorporating author feedback.""",\n    """Once the book content is finalized, generate front and back matter like an introduction, acknowledgments, references, and index."""\n]

### Configure AI Agents\nSet up the AI agents that will assist with the various book writing tasks.\n

In [ ]:
outliner = autogen.AssistantAgent(\n    name="Outliner",\n    llm_config=llm_config,\n)\n\ndrafter = autogen.AssistantAgent(\n    name="Drafter",\n    llm_config=llm_config,\n)\n\neditor = autogen.AssistantAgent(\n    name="Editor",\n    llm_config=llm_config,\n    system_message="""\n        You are a professional editor, known for \n        your keen eye for detail and constructive feedback.\n        You help refine and polish written content.\n        """,\n)

In [ ]:
user = autogen.UserProxyAgent(\n    name="Author",\n    human_input_mode="ALWAYS",\n    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),\n    code_execution_config={\n        "last_n_messages": 1,\n        "work_dir": "book",\n        "use_docker": False,\n    },  \n)\n\nchat_results = await user.a_initiate_chats(  # noqa: F704\n    [\n        {\n            "chat_id": 1,\n            "recipient": outliner,\n            "message": book_tasks[0],\n            "silent": False,\n            "summary_method": "reflection_with_llm",\n        },\n        {\n            "chat_id": 2,\n            "prerequisites": [1],\n            "recipient": user,\n            "message": book_tasks[1],\n            "silent": False,\n            "summary_method": "reflection_with_llm",\n        },\n        {\n            "chat_id": 3,\n            "prerequisites": [2], \n            "recipient": outliner,\n            "message": book_tasks[2],\n            "silent": False,\n            "summary_method": "reflection_with_llm",\n        },\n        {\n            "chat_id": 4,\n            "prerequisites": [3],\n            "recipient": user,\n            "message": book_tasks[3],\n            "silent": False,\n            "summary_method": "reflection_with_llm",\n        },\n        {\n            "chat_id": 5,\n            "prerequisites": [4],\n            "recipient": drafter,\n            "message": book_tasks[4],\n            "silent": False,\n            "summary_method": "reflection_with_llm",\n        },\n        {\n            "chat_id": 6,\n            "prerequisites": [5],\n            "recipient": user,\n            "message": book_tasks[5],\n            "silent": False,\n            "summary_method": "reflection_with_llm",\n        },\n        {\n            "chat_id": 7,\n            "prerequisites": [6],\n            "recipient": drafter,\n            "message": book_tasks[6],\n            "silent": False,\n            "summary_method": "reflection_with_llm",\n        },\n        {\n            "chat_id": 8,\n            "prerequisites": [7],\n            "recipient": user,\n            "message": book_tasks[7],\n            "silent": False,\n            "summary_method": "reflection_with_llm",\n        },\n        {\n            "chat_id": 9,\n            "prerequisites": [8],\n            "recipient": user,\n            "message": book_tasks[8],\n            "silent": False,\n            "summary_method": "reflection_with_llm",\n        },\n        {\n            "chat_id": 10,\n            "prerequisites": [9],\n            "recipient": user,\n            "message": book_tasks[9],\n            "silent": False,\n            "summary_method": "reflection_with_llm",\n        },\n        {\n            "chat_id": 11,\n            "prerequisites": [10],\n            "recipient": outliner,\n            "message": book_tasks[9],\n            "silent": False,\n            "summary_method": "reflection_with_llm",\n        }\n    ]\n)

#### Check chat results\nThe `initiate_chat` method returns a `ChatResult` object, which is a dataclass object storing information about the chat. Currently, it includes the following attributes:\n\n- `chat_history`: a list of chat history.\n- `summary`: a string of chat summary. A summary is only available if a summary_method is provided when initiating the chat.\n- `cost`: a tuple of (total_cost, total_actual_cost), where total_cost is a dictionary of cost information, and total_actual_cost is a dictionary of information on the actual incurred cost with cache.\n- `human_input`: a list of strings of human inputs solicited during the chat. (Note that since we are setting `human_input_mode` to `NEVER` in this notebook, this list is always empty.)

In [ ]:
for i, chat_res in chat_results.items():\n    print(f"*****{i}th chat*******:")\n    print(chat_res.summary)\n    print("Human input in the middle:", chat_res.human_input)\n    print("Conversation cost: ", chat_res.cost)\n    print("\n\n")